In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googleapis/langchain-google-cloud-sql-pg-python/blob/main/samples/langchain_quick_start.ipynb)

---
# **Introduction**

In this codelab, you'll learn how to create a powerful interactive generative AI application using Retrieval Augmented Generation powered by [Cloud SQL for PostgreSQL](https://cloud.google.com/sql/docs/postgres) and [LangChain](https://www.langchain.com/). We will be creating an application grounded in a [Netflix Movie dataset](https://www.kaggle.com/datasets/shivamb/netflix-shows), allowing you to interact with movie data in exciting new ways.

## Prerequisites

* A basic understanding of the Google Cloud Console
* Basic skills in command line interface and Google Cloud shell
* Basic python knowledge

## What you'll learn

* How to deploy a Cloud SQL for PostgreSQL instance
* How to use Cloud SQL for PostgreSQL as a DocumentLoader
* How to use Cloud SQL for PostgreSQL as a VectorStore
* How to use Cloud SQL for PostgreSQL for ChatMessageHistory storage

## What you'll need
* A Google Cloud Account and Google Cloud Project
* A web browser such as [Chrome](https://www.google.com/chrome/)

# **Setup and Requirements**

In the following instructions you will learn to:
1. Install required dependencies for our application
2. Set up authentication for our project
3. Set up a Cloud SQL for PostgreSQL Instance
4. Import the data used by our application

## Install dependencies
First you will need to install the dependencies needed to run this demo app.

In [10]:
%pip install google-auth==2.27.0
%pip install --upgrade --quiet langchain-google-cloud-sql-pg langchain-google-vertexai langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.8/186.8 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.36.0
    Uninstalling google-auth-2.36.0:
      Successfully uninstalled google-auth-2.36.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cloud-sql-python-connector 1.13.0 requires google-auth>=2.28.0, but you have google-auth 2.27.0 which is incompatible.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.27.0, but you have google-auth 2.36.0 which is incompatible.


**Colab only:** Uncomment the following cell to restart the kernel or use the button to restart the kernel. For Vertex AI Workbench you can restart the terminal using the button on top.

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## Authenticate to Google Cloud within Colab
In order to access your Google Cloud Project from this notebook, you will need to Authenticate as an IAM user.

In [2]:
from google.colab import auth

auth.authenticate_user()

## Connect Your Google Cloud Project
Time to connect your Google Cloud Project to this notebook so that you can leverage Google Cloud from within Colab.

In [1]:
# @markdown Please fill in the value below with your GCP project ID and then run the cell.

# Please fill in these values.
project_id = "sanguine-orb-441020-p6"  # @param {type:"string"}

# Quick input validations.
assert project_id, "⚠️ Please provide a Google Cloud project ID"

# Configure gcloud.
!gcloud config set project {project_id}

Updated property [core/project].


## Configure Your Google Cloud Project

Configure the following in your Google Cloud Project.

1. IAM principal (user, service account, etc.) with the [Cloud SQL Client][client-role] role. The user logged into this notebook will be used as the IAM principal and will be granted the Cloud SQL Client role.

[client-role]: https://cloud.google.com/sql/docs/mysql/roles-and-permissions

In [3]:
current_user = !gcloud auth list --filter=status:ACTIVE --format="value(account)"
!gcloud projects add-iam-policy-binding {project_id} \
  --member=user:{current_user[0]} \
  --role="roles/cloudsql.client"

Updated IAM policy for project [sanguine-orb-441020-p6].
bindings:
- members:
  - user:suhanimitra20@gmail.com
  role: roles/cloudsql.client
- members:
  - user:andrewsasamori@gmail.com
  role: roles/cloudsql.instanceUser
- members:
  - serviceAccount:service-232809154320@compute-system.iam.gserviceaccount.com
  role: roles/compute.serviceAgent
- members:
  - serviceAccount:232809154320-compute@developer.gserviceaccount.com
  - serviceAccount:232809154320@cloudservices.gserviceaccount.com
  role: roles/editor
- members:
  - user:andrewsasamori@gmail.com
  - user:noamargolin12@gmail.com
  - user:suhanimitra20@gmail.com
  role: roles/owner
- members:
  - user:jesalamy@gmail.com
  role: roles/resourcemanager.projectOwnerInvitee
- members:
  - serviceAccount:seniordesignspeechtotext@sanguine-orb-441020-p6.iam.gserviceaccount.com
  role: roles/speech.client
etag: BwYm5xzquf0=
version: 1


2. Enable the APIs for Cloud SQL and Vertex AI within your project.

In [4]:
# Enable GCP services
!gcloud services enable sqladmin.googleapis.com aiplatform.googleapis.com

Operation "operations/acat.p2-232809154320-74e1a2c0-663f-403d-815a-9227596feb13" finished successfully.


## Set up Cloud SQL
You will need a **Postgres** Cloud SQL instance for the following stages of this notebook.

### Create a Postgres Instance
Running the below cell will verify the existence of the Cloud SQL instance and or create a new instance and database if one does not exist.

> ⏳ - Creating a Cloud SQL instance may take a few minutes.

In [5]:
#@markdown Please fill in the both the Google Cloud region and name of your Cloud SQL instance. Once filled in, run the cell.

# Please fill in these values.
region = "us-central1" #@param {type:"string"}
instance_name = "langchain-quickstart-instance" #@param {type:"string"}
database_name = "ec463-temp-database" #@param {type:"string"}
password = input("Please provide a password to be used for 'postgres' database user: ")

# Quick input validations.
assert region, "⚠️ Please provide a Google Cloud region"
assert instance_name, "⚠️ Please provide the name of your instance"
assert database_name, "⚠️ Please provide the name of your database_name"

# check if Cloud SQL instance exists in the provided region
database_version = !gcloud sql instances describe {instance_name} --format="value(databaseVersion)"
if database_version[0].startswith("POSTGRES"):
  print("Found existing Postgres Cloud SQL Instance!")
else:
  print("Creating new Cloud SQL instance...")
  !gcloud sql instances create {instance_name} --database-version=POSTGRES_15 \
    --region={region} --cpu=1 --memory=4GB --root-password={password} \
    --database-flags=cloudsql.iam_authentication=On
  !gcloud sql databases create {database_name} --instance={instance_name}


databases = !gcloud sql databases list --instance={instance_name} --format="value(name)"
if database_name not in databases:
  !gcloud sql databases create {database_name} --instance={instance_name}

Please provide a password to be used for 'postgres' database user: ec463
Creating new Cloud SQL instance...
Created [https://sqladmin.googleapis.com/sql/v1beta4/projects/sanguine-orb-441020-p6/instances/langchain-quickstart-instance].
NAME                           DATABASE_VERSION  LOCATION       TIER              PRIMARY_ADDRESS  PRIVATE_ADDRESS  STATUS
langchain-quickstart-instance  POSTGRES_15       us-central1-a  db-custom-1-4096  34.171.54.20     -                RUNNABLE
Created database [ec463-temp-database].
instance: langchain-quickstart-instance
name: ec463-temp-database
project: sanguine-orb-441020-p6


## Import data to your database

Now that you have your database, you will need to import data! We will be using a [Netflix Dataset from Kaggle](https://www.kaggle.com/datasets/shivamb/netflix-shows). Here is what the data looks like:

| show_id | type    | title                | director         | cast                                                                                                                                                  | country       | date_added        | release_year | rating | duration  | listed_in                                    | description                                                                                                                                                                           |
|---------|---------|----------------------|------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------|---------------|-------------------|--------------|--------|-----------|----------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| s1      | Movie   | Dick Johnson Is Dead | Kirsten Johnson  |                                                                                                                                                        | United States | September 25, 2021 | 2020         | PG-13  | 90 min    | Documentaries                                | As her father nears the end of his life, filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable.                              |
| s2      | TV Show | Blood & Water        |                  | Ama Qamata, Khosi Ngema, Gail Mabalane, Thabang Molaba, Dillon Windvogel, Natasha Thahane, Arno Greeff, Xolile Tshabalala, Getmore Sithole, Cindy Mahlangu, Ryle De Morny, Greteli Fincham, Sello Maake Ka-Ncube, Odwa Gwanya, Mekaila Mathys, Sandi Schultz, Duane Williams, Shamilla Miller, Patrick Mofokeng | South Africa  | September 24, 2021 | 2021         | TV-MA  | 2 Seasons | International TV Shows, TV Dramas, TV Mysteries | After crossing paths at a party, a Cape Town teen sets out to prove whether a private-school swimming star is her sister who was abducted at birth.                                   |
| s3      | TV Show | Ganglands            | Julien Leclercq  | Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabiha Akkari, Sofia Lesaffre, Salim Kechiouche, Noureddine Farihi, Geert Van Rampelberg, Bakary Diombera                                   |               | September 24, 2021 | 2021         | TV-MA  | 1 Season  | Crime TV Shows, International TV Shows, TV Action & Adventure | To protect his family from a powerful drug lord, skilled thief Mehdi and his expert team of robbers are pulled into a violent and deadly turf war.                                     |
| s4      | TV Show | Jailbirds New Orleans |                  |                                                                                                                                                        |               | September 24, 2021 | 2021         | TV-MA  | 1 Season  | Docuseries, Reality TV                        | Feuds, flirtations and toilet talk go down among the incarcerated women at the Orleans Justice Center in New Orleans on this gritty reality series.                                   |
| s5      | TV Show | Kota Factory         |                  | Mayur More, Jitendra Kumar, Ranjan Raj, Alam Khan, Ahsaas Channa, Revathi Pillai, Urvi Singh, Arun Kumar                                                 | India        | September 24, 2021 | 2021         | TV-MA  | 2 Seasons | International TV Shows, Romantic TV Shows, TV Comedies | In a city of coaching centers known to train India’s finest collegiate minds, an earnest but unexceptional student and his friends navigate campus life. |


You won't need to directly load the csv data into your database. Instead we prepared a table, "netflix_titles", in the format of a `.sql` file for Postgres. You can easily import the table into your database with one `gcloud` command.

In [6]:
# Import the Netflix titles table using gcloud command
import_command_output = !gcloud sql import sql {instance_name} gs://cloud-samples-data/langchain/cloud-sql/postgres/first_five_netflix_titles.sql --database={database_name} --quiet

if "Imported data" in str(import_command_output):
    print(import_command_output)
elif "already exists" in str(import_command_output):
    print("Did not import because the table already existed.")
else:
    raise Exception(f"The import seems failed:\n{import_command_output}")

['Imported data from [gs://cloud-samples-data/langchain/cloud-sql/postgres/first_five_netflix_titles.sql] into [https://sqladmin.googleapis.com/sql/v1beta4/projects/sanguine-orb-441020-p6/instances/langchain-quickstart-instance].']


# **Use case 1: Cloud SQL for PostgreSQL as a document loader**

Now that you have data in your database, you are ready to use Cloud SQL for PostgreSQL as a [document loader](https://python.langchain.com/docs/modules/data_connection/document_loaders/). This means we will pull data from the database and load it into memory as documents. We can then feed these documents into the vector store.

Next let's connect to our CloudSQL PostgreSQL instance using the PostgresEngine class.

In [11]:
from langchain_google_cloud_sql_pg import PostgresLoader, PostgresEngine, Column

pg_engine = PostgresEngine.from_instance(
    project_id=project_id,
    instance=instance_name,
    region=region,
    database=database_name,
    user="postgres",
    password=password,
)

Once we initialize a PostgresEngine object, we can pass it into the PostgresLoader to connect to a specific database. As you can see we also pass in a query, table_name and a list of columns. The query tells the loader what query to use to pull data. The "content_columns" argument refers to the columns that will be used as "content" in the document object we will later construct. The rest of the columns in that table will become the "metadata" associated with the documents.

In [12]:
table_name = "netflix_titles"
content_columns = ["title", "director", "cast", "description"]
loader = await PostgresLoader.create(
    engine=pg_engine,
    query=f"SELECT * FROM {table_name};",
    content_columns=content_columns,
)

Then let's run the function to pull our documents from out database using our document loader. You can see the first 5 documents from the database here. Nice, you just used CloudSQL for PostgreSQL as a document loader!

In [13]:
documents = await loader.aload()
print(f"Loaded {len(documents)} from the database. 5 Examples:")
for doc in documents[:5]:
    print(doc)

Loaded 8802 from the database. 5 Examples:
page_content='Midnight Mass Mike Flanagan Kate Siegel, Zach Gilford, Hamish Linklater, Henry Thomas, Kristin Lehman, Samantha Sloyan, Igby Rigney, Rahul Kohli, Annarah Cymone, Annabeth Gish, Alex Essoe, Rahul Abburi, Matt Biedel, Michael Trucco, Crystal Balint, Louis Oliver The arrival of a charismatic young priest brings glorious miracles, ominous mysteries and renewed religious fervor to a dying town desperate to believe.' metadata={'show_id': 's6', 'type': 'TV Show', 'country': None, 'date_added': 'September 24, 2021', 'release_year': 2021, 'rating': 'TV-MA', 'duration': '1 Season', 'listed_in': 'TV Dramas, TV Horror, TV Mysteries'}
page_content='My Little Pony: A New Generation Robert Cullen, José Luis Ucha Vanessa Hudgens, Kimiko Glenn, James Marsden, Sofia Carson, Liza Koshy, Ken Jeong, Elizabeth Perkins, Jane Krakowski, Michael McKean, Phil LaMarr Equestria's divided. But a bright-eyed hero believes Earth Ponies, Pegasi and Unicorns sho

# **Use case 2: Cloud SQL for PostgreSQL as Vector Store**

Now, let's learn how to put all of the documents we just loaded into a [vector store](https://python.langchain.com/docs/modules/data_connection/vectorstores/) so that we can use vector search to answer our user's questions!

### Create Your Vector Store table

Based on the documents that we loaded before, we want to create a table with a vector column as our vector store. We will start it by intializing a vector table by calling the `init_vectorstore_table` function from our `engine`. As you can see we list all of the columns for our metadata. We also specify a vector size, 768, that corresponds with the length of the vectors computed by the model our embeddings service uses, Vertex AI's textembedding-gecko.


In [21]:
from langchain_google_cloud_sql_pg import PostgresEngine, Column

sample_vector_table_name = "movie_vector_table_samples"

pg_engine = PostgresEngine.from_instance(
    project_id=project_id,
    instance=instance_name,
    region=region,
    database=database_name,
    user="postgres",
    password=password,
)

pg_engine.init_vectorstore_table(
    sample_vector_table_name,
    vector_size=768,
    metadata_columns=[
        Column("show_id", "VARCHAR", nullable=True),
        Column("type", "VARCHAR", nullable=True),
        Column("country", "VARCHAR", nullable=True),
        Column("date_added", "VARCHAR", nullable=True),
        Column("release_year", "INTEGER", nullable=True),
        Column("rating", "VARCHAR", nullable=True),
        Column("duration", "VARCHAR", nullable=True),
        Column("listed_in", "VARCHAR", nullable=True),
    ],
    overwrite_existing=True,  # Enabling this will recreate the table if exists.
)

### Try inserting the documents into the vector table

Now we will create a vector_store object backed by our vector table in the Cloud SQL database. Let's load the data from the documents to the vector table. Note that for each row, the embedding service will be called to compute the embeddings to store in the vector table. Pricing details can be found [here](https://cloud.google.com/vertex-ai/pricing).

In [18]:
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_cloud_sql_pg import PostgresVectorStore

# Initialize the embedding service. In this case we are using version 003 of Vertex AI's textembedding-gecko model. In general, it is good practice to specify the model version used.
embeddings_service = VertexAIEmbeddings(
    model_name="textembedding-gecko@003", project=project_id
)

vector_store = PostgresVectorStore.create_sync(
    engine=pg_engine,
    embedding_service=embeddings_service,
    table_name=sample_vector_table_name,
    metadata_columns=[
        "show_id",
        "type",
        "country",
        "date_added",
        "release_year",
        "duration",
        "listed_in",
    ],
)

Now let's try to put the documents data into the vector table. Here is a code example to load the first 5 documents in the list.

In [19]:
import uuid

docs_to_load = documents[:5]

# ! Uncomment the following line to load all 8,800+ documents to the database vector table with calling the embedding service.
# docs_to_load = documents

ids = [str(uuid.uuid4()) for i in range(len(docs_to_load))]
vector_store.add_documents(docs_to_load, ids)

['6179d68a-e151-477a-968f-bf867f9516d9',
 'e1f406ad-1fd1-442a-8b1d-6b0665526749',
 '80a32cec-d98e-43e3-b3ad-ac4fe2617155',
 'f45a44d8-76ba-4510-a706-b5f031fe04c0',
 '4169497f-6574-4df7-a5fa-a15ace24af0f']

### Import the rest of your data into your vector table

You don't have to call the embedding service 8,800 times to load all the documents for the demo. Instead, we have prepared a table with the all 8,800+ rows with pre-computed embeddings in a `.sql` file. Again, let's import to our DB using `gcloud` command.

It will restore the `.sql` file to a table with vectors called `movie_vector_table`.

In [20]:
# Import the netflix titles with vector table using gcloud command
import_command_output = !gcloud sql import sql {instance_name} gs://cloud-samples-data/langchain/cloud-sql/postgres/netflix_titles_vector_table.sql --database={database_name} --quiet

if "Imported data" in str(import_command_output):
    print(import_command_output)
elif "already exists" in str(import_command_output):
    print("Did not import because the table already existed.")
else:
    raise Exception(f"The import seems failed:\n{import_command_output}")

['Imported data from [gs://cloud-samples-data/langchain/cloud-sql/postgres/netflix_titles_vector_table.sql] into [https://sqladmin.googleapis.com/sql/v1beta4/projects/sanguine-orb-441020-p6/instances/langchain-quickstart-instance].']


# **Use case 3: Cloud SQL for PostgreSQL as Chat Memory**

Next we will add chat history (called [“memory” in the context of LangChain](https://python.langchain.com/docs/modules/memory/)) to our application so the LLM can retain context and information across multiple interactions, leading to more coherent and sophisticated conversations or text generation. We can use Cloud SQL for PostgreSQL as “memory” storage in our application so that the LLM can use context from prior conversations to better answer the user’s prompts. First let's initialize Cloud SQL for PostgreSQL as memory storage.

In [24]:
from langchain_google_cloud_sql_pg import PostgresChatMessageHistory, PostgresEngine

message_table_name = "message_store"

pg_engine.init_chat_history_table(table_name=message_table_name)

chat_history = PostgresChatMessageHistory.create_sync(
    pg_engine,
    session_id="my-test-session",
    table_name=message_table_name,
)

Here is an example of how you would add a user message and how you would add an ai message.

In [25]:
chat_history.add_user_message("Hi!")
chat_history.add_ai_message("Hello there. I'm a model and am happy to help!")

chat_history.messages

[HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hello there. I'm a model and am happy to help!", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hello there. I'm a model and am happy to help!", additional_kwargs={}, response_metadata={})]

# **PostgresConversational RAG Chain backed by Cloud SQL **

So far we've tested with using Cloud SQL for PostgreSQL as document loader, Vector Store and Chat Memory. Now let's use it in the `ConversationalRetrievalChain`.

We will build a chat bot that can answer movie related questions based on the vector search results.

First let's initialize all of our PostgresSQLEngine object to use as a connection in our vector store and chat_history.

In [27]:
from langchain_google_vertexai import VertexAIEmbeddings, VertexAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationSummaryBufferMemory
from langchain_core.prompts import PromptTemplate
from langchain_google_cloud_sql_pg import (
    PostgresEngine,
    PostgresVectorStore,
    PostgresChatMessageHistory,
)

# Initialize the embedding service
embeddings_service = VertexAIEmbeddings(
    model_name="textembedding-gecko@latest", project=project_id
)

# Initialize the engine
pg_engine = PostgresEngine.from_instance(
    project_id=project_id,
    instance=instance_name,
    region=region,
    database=database_name,
    user="postgres",
    password=password,
)

# Initialize the Vector Store
vector_table_name = "movie_vector_table"
vector_store = PostgresVectorStore.create_sync(
    engine=pg_engine,
    embedding_service=embeddings_service,
    table_name=vector_table_name,
    metadata_columns=[
        "show_id",
        "type",
        "country",
        "date_added",
        "release_year",
        "duration",
        "listed_in",
    ],
)

# Initialize the PostgresChatMessageHistory
chat_history = PostgresChatMessageHistory.create_sync(
    pg_engine,
    session_id="my-test-session",
    table_name="message_store",
)

Let's create a prompt for the LLM. Here we can add instructions specific to our application, such as "Don't make things up".

In [28]:
# Prepare some prompt templates for the ConversationalRetrievalChain
prompt = PromptTemplate(
    template="""Use all the information from the context and the conversation history to answer new question. If you see the answer in previous conversation history or the context. \
Answer it with clarifying the source information. If you don't see it in the context or the chat history, just say you \
didn't find the answer in the given data. Don't make things up.

Previous conversation history from the questioner. "Human" was the user who's asking the new question. "Assistant" was you as the assistant:
```{chat_history}
```

Vector search result of the new question:
```{context}
```

New Question:
```{question}```

Answer:""",
    input_variables=["context", "question", "chat_history"],
)
condense_question_prompt_passthrough = PromptTemplate(
    template="""Repeat the following question:
{question}
""",
    input_variables=["question"],
)

Now let's use our vector store as a retreiver. Retreiver's in Langchain allow us to literally "retrieve" documents.

In [29]:
# Initialize retriever, llm and memory for the chain
retriever = vector_store.as_retriever(
    search_type="mmr", search_kwargs={"k": 5, "lambda_mult": 0.8}
)

Now let's initialize our LLM, in this case we are using Vertex AI's "gemini-pro".

In [30]:
llm = VertexAI(model_name="gemini-pro", project=project_id)

We clear our chat history, so that our application starts without any prior context to other conversations we have had with the application.

In [31]:
chat_history.clear()

memory = ConversationSummaryBufferMemory(
    llm=llm,
    chat_memory=chat_history,
    output_key="answer",
    memory_key="chat_history",
    return_messages=True,
)

<ipython-input-31-b6e34cd98860>:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(


Now let's create a conversational retrieval chain. This will allow the LLM to use chat history in it's responses, meaning we can ask it follow up questions to our questions instead of having to start from scratch after each inquiry.

In [32]:
# create the ConversationalRetrievalChain
rag_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    verbose=False,
    memory=memory,
    condense_question_prompt=condense_question_prompt_passthrough,
    combine_docs_chain_kwargs={"prompt": prompt},
)

# ask some questions
q = "What movie was Brad Pitt in?"
ans = rag_chain({"question": q, "chat_history": chat_history})["answer"]
print(f"Question: {q}\nAnswer: {ans}\n")

q = "How about Jonny Depp?"
ans = rag_chain({"question": q, "chat_history": chat_history})["answer"]
print(f"Question: {q}\nAnswer: {ans}\n")

q = "Are there movies about animals?"
ans = rag_chain({"question": q, "chat_history": chat_history})["answer"]
print(f"Question: {q}\nAnswer: {ans}\n")

# browser the chat history
chat_history.messages

<ipython-input-32-8710e06fe420>:13: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ans = rag_chain({"question": q, "chat_history": chat_history})["answer"]


Question: What movie was Brad Pitt in?
Answer: According to the information you provided, Brad Pitt has starred in the following movies:

* **Inglourious Basterds** (2009)
* **By the Sea** (2015)
* **Killing Them Softly** (2012)
* **Babel** (2006)

Would you like to know more about any of these movies? 


Question: How about Jonny Depp?
Answer: I found information about Johnny Depp's movies in the previous conversation. However, I didn't find any information about Brad Pitt's movies with Johnny Depp. 

Do you have any other questions? 

Question: Are there movies about animals?
Answer: Yes, there are many movies about animals. Some popular examples include:

* **If I Were an Animal:** This documentary follows a group of children as they explore the lives of different animals.
* **The Animal People:** This documentary follows animal rights protesters as they try to shut down a facility that conducts animal testing.
* **Open Season:** This animated film tells the story of a domesticated 

[HumanMessage(content='What movie was Brad Pitt in?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='According to the information you provided, Brad Pitt has starred in the following movies:\n\n* **Inglourious Basterds** (2009)\n* **By the Sea** (2015)\n* **Killing Them Softly** (2012)\n* **Babel** (2006)\n\nWould you like to know more about any of these movies? \n', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='How about Jonny Depp?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="I found information about Johnny Depp's movies in the previous conversation. However, I didn't find any information about Brad Pitt's movies with Johnny Depp. \n\nDo you have any other questions? ", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Are there movies about animals?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Yes, there are many movies about animals. Some popular examples include:\n\n* **If I Were